In [2]:
import pickle
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import os

In [16]:
# batchFile='/home/srajaram/preference_videos/humanlabels/hopper-medium-expert-v2/batch_numq100_len998_s3407.pkl'
# with open(batchFile, 'rb') as f:
#     batchDataset = pickle.load(f)

# batchFile='/home/srajaram/preference_videos/humanlabels/hopper-medium-expert-v2/batch_numq100_len100_s3407.pkl'
# with open(batchFile, 'rb') as f:
#     dataset = pickle.load(f)

batchFile='/home/srajaram/preference_videos/humanlabels/hopper-medium-replay-v2/batch_numq500_len100_s3407.pkl'
with open(batchFile, 'rb') as f:
    dataset = pickle.load(f)

# batchFile='/home/srajaram/preference_videos/test/hopper-medium-expert-v2/batch_numq10_len400_s3407.pkl'
# with open(batchFile, 'rb') as f:
#     longqueries = pickle.load(f)

# batchFile='/home/srajaram/preference_videos/testCorr/hopper-medium-expert-v2/batch_numq10_len400_s3407.pkl'
# with open(batchFile, 'rb') as f:
#     longqueriesCorr = pickle.load(f)

### Understand Query Functions

In [100]:
from tqdm import tqdm
import os
import numpy as np
import pickle

import gym
import imageio
import jax
import numpy as np
from absl import app, flags
from tqdm import tqdm, trange

import d4rl

#########IMPORTED LOCALLY############
######REGISTERING ENVIRONMENTS#####


<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


In [104]:

def qlearning_mujoco_dataset(env, dataset=None, terminate_on_end=False, **kwargs):
    """
    Returns datasets formatted for use by standard Q-learning algorithms,
    with observations, actions, next_observations, rewards, and a terminal
    flag.
    Args:
        env: An OfflineEnv object.
        dataset: An optional dataset to pass in for processing. If None,
            the dataset will default to env.get_dataset()
        terminate_on_end (bool): Set done=True on the last timestep
            in a trajectory. Default is False, and will discard the
            last timestep in each trajectory.
        **kwargs: Arguments to pass to env.get_dataset().
    Returns:
        A dictionary containing keys:
            observations: An N x dim_obs array of observations.
            actions: An N x dim_action array of actions.
            next_observations: An N x dim_obs array of next observations.
            rewards: An N-dim float array of rewards.
            terminals: An N-dim boolean array of "done" or episode termination flags.
    """
    if dataset is None:
        dataset = env.get_dataset(**kwargs)

    N = dataset["rewards"].shape[0]
    obs_ = []
    next_obs_ = []
    action_ = []
    reward_ = []
    done_ = []
    xy_ = []
    done_bef_ = []

    qpos_ = []
    qvel_ = []

    # The newer version of the dataset adds an explicit
    # timeouts field. Keep old method for backwards compatability.
    use_timeouts = False
    if "timeouts" in dataset:
        use_timeouts = True

    episode_step = 0
    for i in range(N - 1):
        obs = dataset["observations"][i].astype(np.float32)
        new_obs = dataset["observations"][i + 1].astype(np.float32)
        action = dataset["actions"][i].astype(np.float32)
        reward = dataset["rewards"][i].astype(np.float32)
        done_bool = bool(dataset["terminals"][i]) or episode_step == env._max_episode_steps - 1
        xy = dataset["infos/qpos"][i][:2].astype(np.float32)

        qpos = dataset["infos/qpos"][i]
        qvel = dataset["infos/qvel"][i]

        if use_timeouts:
            final_timestep = dataset["timeouts"][i]
            next_final_timestep = dataset["timeouts"][i + 1]
        else:
            final_timestep = episode_step == env._max_episode_steps - 1
            next_final_timestep = episode_step == env._max_episode_steps - 2

        done_bef = bool(next_final_timestep)

        if (not terminate_on_end) and final_timestep:
            # Skip this transition and don't apply terminals on the last step of an episode
            episode_step = 0
            continue
        if done_bool or final_timestep:
            episode_step = 0

        obs_.append(obs)
        next_obs_.append(new_obs)
        action_.append(action)
        reward_.append(reward)
        done_.append(done_bool)
        xy_.append(xy)
        done_bef_.append(done_bef)

        qpos_.append(qpos)
        qvel_.append(qvel)
        episode_step += 1

    return {
        "observations": np.array(obs_),
        "actions": np.array(action_),
        "next_observations": np.array(next_obs_),
        "rewards": np.array(reward_),
        "terminals": np.array(done_),
        "xys": np.array(xy_),
        "dones_bef": np.array(done_bef_),
        "qposes": np.array(qpos_),
        "qvels": np.array(qvel_),
    }

In [101]:
def set_seed(env, seed):
    np.random.seed(seed)
    env.seed(seed)
    env.observation_space.seed(seed)
    env.action_space.seed(seed)

In [98]:
query_path='/home/srajaram/PreferenceTransformer/human_label/'
env_name='hopper-medium-expert-v2'
base_path = os.path.join(query_path, env_name)
human_indices_2_file, human_indices_1_file, label_file = sorted(os.listdir(base_path))
with open(os.path.join(base_path, human_indices_1_file), "rb") as fp:   # Unpickling
    human_indices = pickle.load(fp)
with open(os.path.join(base_path, human_indices_2_file), "rb") as fp:   # Unpickling
    human_indices_2 = pickle.load(fp)
with open(os.path.join(base_path, label_file), "rb") as fp:   # Unpickling
    human_labels = pickle.load(fp)

In [105]:
saved_indices=[human_indices, human_indices_2]
num_query=10
saved_labels=human_labels
len_query=400
seed=3407
gym_env = gym.make(env_name)

set_seed(gym_env, seed)
dataset = qlearning_mujoco_dataset(gym_env)


/home/srajaram/.local/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/srajaram/.local/lib/python3.10/site-packages/gym/core.py:172: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed) instead.
  deprecation(
load datafile: 100%|██████████| 9/9 [00:03<00:00,  2.85it/s]


In [111]:
saved_indices[1][90]

680608

In [115]:
dataset.keys()

dict_keys(['observations', 'actions', 'next_observations', 'rewards', 'terminals', 'xys', 'dones_bef', 'qposes', 'qvels'])

In [108]:
query_range = np.arange(len(saved_labels) - num_query, len(saved_labels))
for query_count, i in enumerate(tqdm(query_range, desc="get queries from saved indices")):
    temp_count = 0
    while(temp_count < 2):                
        start_idx = saved_indices[temp_count][i]
        end_idx = start_idx + len_query

        reward_seq = dataset['rewards'][start_idx:end_idx]
        obs_seq = dataset['observations'][start_idx:end_idx]
        next_obs_seq = dataset['next_observations'][start_idx:end_idx]
        act_seq = dataset['actions'][start_idx:end_idx]
        timestep_seq = np.arange(1, len_query + 1)

        print(query_count)
        print(i)
        print(start_idx)
        print(end_idx)
        print("#############")

        # if temp_count == 0:
        #     total_reward_seq_1[query_count] = reward_seq
        #     total_obs_seq_1[query_count] = obs_seq
        #     total_next_obs_seq_1[query_count] = next_obs_seq
        #     total_act_seq_1[query_count] = act_seq
        #     total_timestep_1[query_count] = timestep_seq
        # else:
        #     total_reward_seq_2[query_count] = reward_seq
        #     total_obs_seq_2[query_count] = obs_seq
        #     total_next_obs_seq_2[query_count] = next_obs_seq
        #     total_act_seq_2[query_count] = act_seq
        #     total_timestep_2[query_count] = timestep_seq
                
        temp_count += 1

get queries from saved indices: 100%|██████████| 10/10 [00:00<00:00, 11834.94it/s]

0
90
365443
365843
#############
0
90
680608
681008
#############
1
91
455060
455460
#############
1
91
262910
263310
#############
2
92
1556756
1557156
#############
2
92
790404
790804
#############
3
93
924268
924668
#############
3
93
741610
742010
#############
4
94
185003
185403
#############
4
94
226570
226970
#############
5
95
1807741
1808141
#############
5
95
126380
126780
#############
6
96
184071
184471
#############
6
96
363840
364240
#############
7
97
126103
126503
#############
7
97
823850
824250
#############
8
98
38993
39393
#############
8
98
534067
534467
#############
9
99
924769
925169
#############
9
99
550661
551061
#############


### Explore Datasets

In [20]:
train_size=.8
use05=False

In [23]:
#agent_list=['unrestricted','restricted']
#unrestricted is agent 0, restricted is agent 1. Our restricted here is preferred, so agIdx=1 for preferred and 0 for not preferred#
#this dataset is created assuming average pooling! No token prepended. 
#truncateGoalSteps=False (not truncating)
datasetTorch={}
for key in dataset:
    datasetTorch[key]=torch.from_numpy(dataset[key])
    
def return_trajs(indices,preferredSuffix,notPreferredSuffix,max_len=1000):
    prefObsKey,prefActionKey='observations'+preferredSuffix,'actions'+preferredSuffix
    notPrefObsKey,notPrefActionKey='observations'+notPreferredSuffix,'actions'+notPreferredSuffix
    prefObs, prefAction= datasetTorch[prefObsKey][indices], datasetTorch[prefActionKey][indices]
    prefAgIdx=torch.zeros(prefObs.shape[:-1])+1
    prefAgIdx=prefAgIdx.unsqueeze(2)

    #taskIdPref=torch.zeros_like(prefAgIdx) #dummy task ID so it works in the existing capacity encoder framework 

    notPrefObs, notPrefAction= datasetTorch[notPrefObsKey][indices], datasetTorch[notPrefActionKey][indices]
    notPrefAgIdx=torch.zeros_like(prefAgIdx)

    #taskIdNotPref=torch.zeros_like(prefAgIdx) #dummy task ID so it works in the existing framework 

    maskShape=prefObs.shape[:-1]
    src_key_padding_mask=torch.zeros(maskShape,dtype=torch.bool).unsqueeze(2) # assumed not truncating, so set to False. Both trajs use the same mask (all False anyway)

    trajsPreferred=torch.cat([prefObs,prefAction,src_key_padding_mask, prefAgIdx],dim=2) #numTrajectories x stepMax(100) x obsDim+actionDim+3 (mask+agentId+taskId)
    trajsNotPreferred=torch.cat([notPrefObs,notPrefAction,src_key_padding_mask,notPrefAgIdx],dim=2)

    all_agent_trajectories=torch.cat([trajsPreferred,trajsNotPreferred],dim=0)
    return all_agent_trajectories

if use05:
    indices10=np.union1d((dataset['labels']==[.5,.5]).all(axis=1).nonzero()[0], (dataset['labels']==[1,0]).all(axis=1).nonzero()[0]) #here we include both trajs where neither is preferred ([.5, .5]) in both agent types
else:
    indices10=(dataset['labels']==[1,0]).all(axis=1).nonzero()[0]
trajs10=return_trajs(indices10,'','_2')

indices01=(dataset['labels']==[0,1]).all(axis=1).nonzero()[0]# #here we only include the [0,1] indices. If we included [.5,.5] again we would double count indices. 
trajs01=return_trajs(indices01,'_2','')

all_trajectories=torch.cat([trajs10,trajs01],dim=0)
all_trajectories=all_trajectories[torch.randperm(all_trajectories.size()[0])]
all_trajectories=all_trajectories.float()
if train_size<1.0:
    train_set, test_set = train_test_split(all_trajectories, train_size=train_size, shuffle=True)
else:
    train_set=all_trajectories



In [199]:
train_set.shape

torch.Size([200, 100, 16])

In [105]:
filepath='/home/srajaram/preference_videos/humanlabels/hopper-medium-expert-v2/train_set_with05.pkl'

In [106]:
with open('/home/srajaram/preference_videos/humanlabels/hopper-medium-expert-v2/train_set_with05.pkl', 'wb') as f:  # open a text file
    pickle.dump(train_set, f)

In [9]:
print(train_set.shape)
print(test_set.shape)
with open('/home/srajaram/preference_videos/humanlabels/hopper-medium-expert-v2/traintestsplit05/train_set_with05.pkl', 'wb') as f:  # open a text file
    pickle.dump(train_set, f)
with open('/home/srajaram/preference_videos/humanlabels/hopper-medium-expert-v2/traintestsplit05/test_set_with05.pkl', 'wb') as f:  # open a text file
    pickle.dump(test_set, f)


torch.Size([160, 100, 16])
torch.Size([40, 100, 16])


In [15]:
print(train_set.shape)
print(test_set.shape)
with open('/home/srajaram/preference_videos/humanlabels/hopper-medium-expert-v2/traintestsplitNo05/train_set_No05.pkl', 'wb') as f:  # open a text file
    pickle.dump(train_set, f)
with open('/home/srajaram/preference_videos/humanlabels/hopper-medium-expert-v2/traintestsplitNo05/test_set_No05.pkl', 'wb') as f:  # open a text file
    pickle.dump(test_set, f)

torch.Size([115, 100, 16])
torch.Size([29, 100, 16])


In [24]:
#hopper medium replay v2
print(train_set.shape)
print(test_set.shape)
with open('/home/srajaram/preference_videos/humanlabels/hopper-medium-replay-v2/traintestsplitNo05/train_set_No05.pkl', 'wb') as f:  # open a text file
    pickle.dump(train_set, f)
with open('/home/srajaram/preference_videos/humanlabels/hopper-medium-replay-v2/traintestsplitNo05/test_set_No05.pkl', 'wb') as f:  # open a text file
    pickle.dump(test_set, f)

torch.Size([494, 100, 16])
torch.Size([124, 100, 16])


### Testing Gym env render

In [2]:
import sys
sys.path.append('/home/srajaram/')
sys.path.append('/home/srajaram/rltransfer')
sys.path.append('/home/srajaram/rltransfer/url_benchmark')
import url_benchmark
from url_benchmark.d4rl_benchmark import D4RLWrapper, D4RLReplayBufferBuilder
from url_benchmark import dmc
from url_benchmark.ContrastiveCapacityEncoderV2pbrl import PositionalEncoding, TransformerModel
from collections import OrderedDict
import gym
import d4rl
width=256
height=256
camera_name='track'



#########IMPORTED LOCALLY############
######REGISTERING ENVIRONMENTS#####


<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


In [2]:
import os
os.getcwd()

'/home/srajaram/PreferenceTransformer/JaxPref'

In [19]:
gym_env.get_normalized_score(3000)

0.9280089738242887

In [21]:
gym_env = gym.make('hopper-medium-expert-v2')
maxSc=gym_env.spec.kwargs['ref_max_score']
minSc=gym_env.spec.kwargs['ref_min_score']

/home/srajaram/.local/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [18]:
gym_env.spec.max_episode_steps

1000

In [8]:
gym_env = gym.make('hopper-medium-expert-v2')
gym_env.reset()
curr_frame = gym_env.sim.render(width=width, height=height, mode="offscreen", camera_name=camera_name)
print(curr_frame.shape)

(256, 256, 3)


/home/srajaram/.local/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [4]:
evalenv=dmc.EnvWrapper(D4RLWrapper(gym.make('hopper-medium-expert-v2')))

evalenv.reset()

frame=evalenv.base_env.base_env.sim.render(width=width, height=height, mode="offscreen", camera_name=camera_name)
print(frame.shape)

AUGMENT TIMESTEP
ExtendedTimeStepD4RL(step_type=<StepType.FIRST: 0>, reward=None, discount=None, observation=array([ 1.25387213e+00, -2.76274593e-03,  1.72436865e-04, -1.10827922e-03,
       -3.18006322e-03, -5.91725644e-04, -4.25065479e-03,  5.38520324e-04,
       -2.26425940e-03,  2.45247530e-03,  1.72598181e-04]), physics=array(0.15), action=array([ 0.703257 ,  0.5275694, -0.969665 ], dtype=float32))
True
(256, 256, 3)


### Make Full Dataset for Decoder

This is a bit convoluted but it makes things compatible with dmc so we don't run into issues later

In [3]:
gym_env = dmc.EnvWrapper(D4RLWrapper(gym.make('hopper-medium-expert-v2'))) ##make dmc compatible environment


/home/srajaram/.local/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [187]:
def create_dataset_fulloffline(gym_env): #This is a bit convoluted but it makes things compatible with dmc so we don't run into issues later
    d4rl_replay_buffer_builder = D4RLReplayBufferBuilder() #use dmc function which creates ReplayBuffer
    replay_storage = d4rl_replay_buffer_builder.prepare_replay_buffer_d4rl(gym_env, OrderedDict(), discount=.99, future=.99, minimum_episode_length=1, ignore_terminals=False)
    actions=replay_storage._storage['action'].copy()
    actions[:,:-1,:]=actions[:,1:,:] #deepmind control suite aligns the actions such that it pairs obs with the action that got you into obs. This is not the action that was taken in obs. Thus we shift accordingly such that obs is paired with action taken in obs
    actions=torch.tensor(actions)
    obs=torch.tensor(replay_storage._storage['observation'].copy())

    maskShape=obs.shape[:-1]
    epLens=torch.tensor(replay_storage._episodes_length.copy()).unsqueeze(1) #this is really the index of the last state, rather than the length (ie episode_length-1)
    columns = torch.arange(maskShape[1]).unsqueeze(0)
    src_key_padding_mask = columns >= epLens #this actually has the masking start one timestep before the last observation. This is because we don't have an action taken at the last observation, so the last timestep observed is not useable for encoding or decoding 
    src_key_padding_mask=src_key_padding_mask.unsqueeze(2)

    batch_indices = torch.arange(maskShape[0])
    obs[batch_indices, epLens.squeeze(), :]=0.0 #because the last obs doesn't have an action to go with it, set it equal to zero so that nothing weird when training model (ie it doesn't try to map the last obs to doing nothing). Of course, the src_key_padding_mask should take care of this, but just to be on the safe side 

    datasetNew=torch.cat([obs,actions,src_key_padding_mask],dim=2)
    return datasetNew

In [188]:
ds=create_dataset_fulloffline(gym_env)
    

load datafile: 100%|██████████| 9/9 [00:03<00:00,  2.96it/s]


In [193]:
filepath='/home/srajaram/preference_videos/hopper-medium-expert-v2/fulldataset.pkl'
with open(filepath, 'wb') as f:  # open a text file
    pickle.dump(ds, f)

### Offline RL Toolkit

In [16]:
import sys
sys.path.append('/home/srajaram/')
sys.path.append('/home/srajaram/rltransfer')
sys.path.append('/home/srajaram/rltransfer/url_benchmark')
sys.path.append('/home/srajaram/rltransfer/OfflineRL-Kit')
import url_benchmark
from url_benchmark.pbrl.CottonDecoderV2pbrl import TransformerForInference
from collections import OrderedDict

import gym
import d4rl

import offlinerlkit
from offlinerlkit.nets import MLP
from offlinerlkit.modules import ActorProb, Critic, TanhDiagGaussian
from offlinerlkit.utils.load_dataset import qlearning_dataset
from offlinerlkit.buffer import ReplayBuffer
from offlinerlkit.utils.logger import Logger, make_log_dirs
from offlinerlkit.policy_trainer import MFPolicyTrainer
from offlinerlkit.policy import CQLPolicy

import PreferenceTransformer
from PreferenceTransformer.dataset_utils import split_into_trajectories    
from tqdm import tqdm, trange
from torch.utils.data import DataLoader
import numpy as np
import torch
import os
import pickle

In [5]:
task='hopper-medium-replay-v2'
env = gym.make(task)
eps= 1e-5
clip_to_eps=True
dataset = d4rl.qlearning_dataset(env)

if clip_to_eps:
    lim = 1 - eps
    dataset['actions'] = np.clip(dataset['actions'], -lim, lim)

dones_float = np.zeros_like(dataset['rewards'])

for i in range(len(dones_float) - 1):
    if np.linalg.norm(dataset['observations'][i + 1] -
                        dataset['next_observations'][i]
                        ) > 1e-5 or dataset['terminals'][i] == 1.0:
        dones_float[i] = 1
    else:
        dones_float[i] = 0

dones_float[-1] = 1
dataset['observations']=dataset['observations'].astype(np.float32)
dataset['actions']=dataset['actions'].astype(np.float32)
dataset['rewards']=dataset['rewards'].astype(np.float32)
dataset['masks']=1.0-dataset['terminals'].astype(np.float32)
dataset['terminals']=dones_float.astype(np.float32)
dataset['next_observations']=dataset['next_observations'].astype(
                        np.float32)

trajs = split_into_trajectories(
    dataset['observations'],
    dataset['actions'],
    dataset['rewards'],
    dataset['masks'], #masks
    dataset['terminals'],
    dataset['next_observations']
)

/home/srajaram/.local/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
split: 100%|██████████| 401598/401598 [00:00<00:00, 604119.71it/s]


In [8]:
max_ep_len=1000
obsDim=dataset['observations'].shape[1]
actionDim=dataset['actions'].shape[1]
dataForEncoder = torch.zeros((len(trajs),max_ep_len,obsDim+actionDim+1)) #the +1 is for the mask 
dataset_idx_mapper = torch.zeros(len(trajs),max_ep_len)

dataset_idx=0
for trj_idx, traj in tqdm(enumerate(trajs), total=len(trajs), desc="chunk trajectories"):
    _obs=torch.zeros((len(traj),obsDim))
    _action=torch.zeros((len(traj),actionDim))
    
    for stepIdx,transitionTuple in enumerate(traj):
        _o, _a, _r, _m, _d, _no=transitionTuple
        _obs[stepIdx,:]=torch.tensor(_o)
        _action[stepIdx,:]=torch.tensor(_a)
        dataset_idx_mapper[trj_idx,stepIdx]=dataset_idx #need this to go back to the buffer of transitions
        dataset_idx+=1
    
    epLen=_obs.shape[0]
    dataForEncoder[trj_idx,:epLen,:obsDim]=_obs
    dataForEncoder[trj_idx,:epLen,obsDim:obsDim+actionDim]=_action
    #src key padding mask: 0s where we have real data, 1 from onwards
    maskCol=dataForEncoder[trj_idx,:epLen,-1]
    columns = torch.arange(max_ep_len)
    src_key_padding_maskCol = columns >= epLen 
    dataForEncoder[trj_idx,:,-1]=src_key_padding_maskCol
    ######################

    

chunk trajectories:   0%|          | 0/2039 [00:00<?, ?it/s]

chunk trajectories: 100%|██████████| 2039/2039 [00:06<00:00, 298.49it/s]


In [31]:
cfg={}
cfg['causal_pool1']=True
cfg['causal_pool2']=False
cfg['alpha']=.01
cfg['beta']=.02
capacityEncoderFilepath='/home/srajaram/rltransfer/exp_local/2025.02.01/_HopperMediumReplayCapacityNo05_smallerdims/103727'
with open(os.path.join(capacityEncoderFilepath,'cfg.pkl'), 'rb') as f:
    capEncCfg = pickle.load(f) 
   

capacity_inference=TransformerForInference(capEncCfg, causal_pool1=cfg['causal_pool1'],causal_pool2=cfg['causal_pool2'])
capacity_inference.eval()

with open(os.path.join(capacityEncoderFilepath,'train_set.pkl'), 'rb') as f:
    train_set = pickle.load(f)
trainDataloader=DataLoader(train_set, batch_size=256, shuffle=False) #creating dataloader only because the relevant function (getEvalLatents) requires it as input, but will just be using the full dataset in that function
with open(os.path.join(capacityEncoderFilepath,'test_set.pkl'), 'rb') as f:
    test_set = pickle.load(f)
if len(test_set)!=0:
    raise ValueError("You should use a capacity encoder model that was trained on the full dataset (so there should be no test set)")

ValueError: You should use a capacity encoder model that was trained on the full dataset (so there should be no test set)

In [22]:
dataForCapacity=dataForEncoder[:,:,:capacity_inference.inputDim] #needed for the capacity latent inference, not distinguishing by agent type
actions=dataForEncoder[:,:,obsDim:obsDim+actionDim] #used for decoder loss
mask=dataForEncoder[:,:,obsDim+actionDim] #padding mask, needed for both the decoder loss and the capacity latent inference 

capacityLatents=capacity_inference.forward(dataForCapacity, src_key_padding_mask=mask)

In [34]:
#get fixed latents from the two classes of agents (preferred and not preferred)
capacityEncoder=url_benchmark.ContrastiveCapacityEncoderV2pbrl.CapacityEncoderV2(capEncCfg,'',train_set=train_set,test_set=[],use_wandb=False) #the train set and test set must be passed in to initialize the class but we don't use it here. 
capacityEncoder.capacity_encoder.load_state_dict(torch.load(os.path.join(capacityEncoderFilepath,'capacityEncoder.pt'), weights_only=True))
capacityEncoder.capacity_encoder.eval()
capacityEncoder=capacityEncoder
allAgentLatents={} #only used in policy eval rollouts
for i in range(len(capacityEncoder.agent_list)):
    latents=capacityEncoder.getEvalLatents(i,trainDataloader).clone().detach()
    if i==0:
        agent='unpreferred'
    if i==1:
        agent='preferred'
    allAgentLatents[agent]=latents

#######Using preloaded train/test sets#######


In [55]:
cos=torch.nn.CosineSimilarity(dim=2)
simWUnpreferred=cos(capacityLatents,allAgentLatents['unpreferred'].repeat(max_ep_len,1))
simWPreferred=cos(capacityLatents,allAgentLatents['preferred'].repeat(max_ep_len,1)) 
new_rewards=cfg['alpha']*simWPreferred-cfg['beta']*simWUnpreferred
new_rewards=new_rewards.detach().numpy() #shape number of trajectories, max_ep_len, this will include rewards for timesteps of a trajectory that were padded on. But we'll take care of that in rematching to the dataset

##### The above was pulled together into the function below

In [17]:
with open(os.path.join('/home/srajaram/rltransfer/exp_local/2025.02.06/HopperMediumReplayCapacityOfflineDataset','offline_dataset_modifiedreward.pkl'), 'rb') as f:
    offlinedataset = pickle.load(f)

In [11]:
import numpy as np

In [1]:
import sys
sys.path.append('/user/rajaram/u13657/rltransfer')
import url_benchmark
from url_benchmark.pbrl.make_offlinedataset import make_offline_dataset
import os
import pickle
import url_benchmark
from url_benchmark.pbrl.CottonDecoderV2pbrl import TransformerForInference
import torch

#########IMPORTED LOCALLY############


<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


######REGISTERING ENVIRONMENTS#####


/opt/conda/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65550) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


In [8]:
dataset['rewards']

array([-0.19817598, -0.98015704, -1.87695922, ..., -0.13831469,
        0.66657294,  0.21992515])

In [7]:
datasetLowRew['rewards']

array([-0.19817601, -0.98015707, -1.87695932, ..., -0.1383147 ,
        0.66657295,  0.21992514])

In [14]:
absolute_differences = np.abs(dataset['rewards'] - datasetLowRew['rewards'])

mean_diff = np.mean(absolute_differences)
std_diff = np.std(absolute_differences)
print(mean_diff)
print(std_diff)

1.7676622322017877e-08
1.8516079241829727e-08


In [42]:
(np.round(dataset['rewards'],3)!=np.round(datasetLowRew['rewards'],3)).nonzero()

(array([ 51550, 124416, 148449, 194485, 224855, 229099, 230182, 333364]),)

In [44]:
print(np.round(dataset['rewards'],3)[51550])
print(dataset['rewards'][51550])

-1.071
-1.071499970303982


In [43]:
print(np.round(datasetLowRew['rewards'],3)[51550])
print(datasetLowRew['rewards'][51550])

-1.072
-1.0715000142157831


In [34]:
dataset_clipped = np.floor(dataset['rewards'] * 1e3) / 1e3
datasetLowRew_clipped = np.floor(datasetLowRew['rewards'] * 1e3) / 1e3
absolute_differences = np.abs(dataset_clipped - datasetLowRew_clipped)

mean_diff = np.mean(absolute_differences)
std_diff = np.std(absolute_differences)
print(mean_diff)
print(std_diff)


2.4900522412961353e-08
4.9899801980514895e-06


In [5]:
cfg={'capacityEncoderFilepath':os.path.join(os.path.expanduser('~'),'rltransfer/exp_local/2025.02.01/_HopperMediumReplayCapacityNo05_smallerdims/103727'), 'task':'hopper-medium-replay-v2', 'max_ep_len':1000}
cfg['causal_pool1']=False
cfg['causal_pool2']=False
cfg['alpha']=.1
cfg['beta']=0.0
datasetLowRew=make_offline_dataset(cfg)

/user/rajaram/u13657/.local/lib/python3.10/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
chunk trajectories: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 2039/2039 [00:08<00:00, 250.65it/s]


#######Using preloaded train/test sets#######
Input train set size 494 and test set size is 0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31


chunk trajectories: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2039/2039 [00:00<00:00, 52689.79it/s]
